# Introduction

Brian would like to see just the splices for some subsets.

How do I figure that out?

In [19]:
import os
import pandas
from pysam import AlignmentFile
from collections import Counter

In [2]:
analysis_dir='/woldlab/castor/home/sau/flowcells/H7CNTBCX2/19915_B9'
bam_file = os.path.join(analysis_dir, '19915_B9-mm10-M4-male_genome.bam')
sj_file = os.path.join(analysis_dir, 'SJ.out.tab')

In [3]:
sj = pandas.read_csv(
    sj_file, sep='\t', header=None, index_col=None,
    names=['chr', 'start', 'stop', 'strand', 'intron', 'annotated', 'unique', 'multi', 'max_overhang']
)

In [4]:
sj.head()

,chr,start,stop,strand,intron,annotated,unique,multi,max_overhang
0,chr10,3366970,3423493,1,1,1,1,0,13
1,chr10,4403991,4407804,2,2,1,1,2,25
2,chr10,4407926,4410712,2,2,1,0,2,14
3,chr10,4410790,4411666,2,2,1,0,6,18
4,chr10,4411732,4413308,2,2,1,2,1,13


In [12]:
sj.shape

(31837, 9)

In [11]:
sj[sj['annotated'] == 0].shape

(1055, 9)

intron  motif:  0:  non-canonical;  1:  GT/AG,  2:  CT/AC,  3:  GC/AG,  4:  CT/GC,  5:AT/AC, 6:  GT/AT

In [16]:
sj[sj['intron'] > 1].shape

(16006, 9)

In [5]:
reads = AlignmentFile(bam_file, 'rb')

In [7]:
for r in reads.fetch('chr10', 3366970, 3423493):
    print(r)

HISEQ:768:H7CNTBCX2:2:1202:2308:89307	0	0	3366956	255	13M56524N37M	-1	-1	50	GTACGGCTGCGAGGAAGAAGAAATGCCAGATGTAGAAATTGACATTGATG	array('B', [35, 35, 34, 35, 35, 40, 39, 39, 40, 40, 40, 40, 40, 40, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 40, 39, 40, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 40, 39, 39, 40, 39, 39, 39, 39])	[('NH', 1), ('HI', 1), ('AS', 47), ('NM', 0), ('MD', '50')]


In [62]:
def count_splices(read):
    splices = 0
    for c in read.cigar:
        if c[0] == 3:
            splices += 1
    return splices
    
def is_spliced(read):
    for c in read.cigar:
        if c[0] == 3:
            return True
    return False

In [61]:
limit = 5
for i, junction in sj.iterrows():
    uniq = 0
    seen_starts = set()
    for r in reads.fetch(junction.chr, junction.start, junction.stop):
        if r.get_tag('NH') == 1 and is_splice(r) and r.pos not in seen_starts:
            uniq += 1
        seen_starts.add(r.pos)
    if uniq != junction['unique']:
        print(i, uniq, junction['unique'], junction['multi'])
        limit -= 1
    if limit <= 0:
        break
print(i, limit)

10 2 1 0
11 2 1 0
14 1 2 0
28 5 7 0
34 12 0 2
34 0


In [47]:
uniq

5

In [57]:
for row in [x for x in reads.fetch(junction.chr, junction.start, junction.stop) if is_splice(x)]:
    if row.get_tag('NH') == 1:
        print(row)


HISEQ:768:H7CNTBCX2:2:2115:19363:9456	16	0	7649120	255	35M13919N15M	-1	-1	50	TTCAAATACTTTATCTGTCTTGATGATTCCATTCTTGCGGAGGTTGTGGA	array('B', [40, 40, 39, 39, 39, 39, 39, 40, 40, 40, 40, 39, 39, 36, 40, 39, 37, 39, 39, 40, 39, 39, 40, 39, 38, 39, 39, 39, 36, 40, 39, 39, 39, 39, 35, 39, 38, 40, 40, 40, 40, 39, 40, 40, 40, 35, 33, 35, 27, 15])	[('NH', 1), ('HI', 1), ('AS', 48), ('NM', 0), ('MD', '50')]
HISEQ:768:H7CNTBCX2:1:2112:7989:53812	16	0	7649128	255	1S27M13919N22M	-1	-1	49	TCTTTATCTGTCTTGATGATTCCATTCTTGCGGAGGTTGTGGATTAGCTC	array('B', [16, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 35, 35, 35, 32, 35])	[('NH', 1), ('HI', 1), ('AS', 47), ('NM', 0), ('MD', '49')]
HISEQ:768:H7CNTBCX2:1:2110:3684:54492	0	0	7667635	255	46M320N4M	-1	-1	50	CCTGCAGACCACAGAGACCTTTGCCACGGGCTCTTGGGACAATGAGGAAA	array('B', [35, 35, 35, 35, 35, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 

In [44]:
row.cigar

[(0, 30), (3, 7255), (0, 20)]

In [49]:
row.get_tags()

[('NH', 1), ('HI', 1), ('AS', 49), ('NM', 0), ('MD', '50')]

In [ ]:
def find_spliced_reads(reads):
    for r in reads:
        if is_spliced(r):
            yield r
            

In [63]:
reads.header